<a href="https://colab.research.google.com/github/NastasiaMazur/BM25_Experiments/blob/main/Simple_inverted_index_and_rank_BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install rank_bm25

In [33]:
#This project made for data-structure class
import time
import os
import string
start = time.time()
#from nltk.corpus import gutenberg
from string import punctuation
from rank_bm25 import BM25Okapi
from nltk.tokenize import wordpunct_tokenize
#from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
nltk.download('gutenberg')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [34]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [35]:
stop_words = stopwords.words('english') # The list with the stopwords
for i in punctuation:
    stop_words.append(i)    # We append at the stopwords list, the punctuation (expecting better results)

# Initialize the inverted index and documents list
the_inverted_index = dict()  # Key = The Words, value = a List with tuples (DocID, Count)
docs = []  # The list to store the content of each book

# Set the path to your Google Drive folder containing the books
mypath = '/content/drive/My Drive/Documents'
books_collection = os.listdir(mypath)

# Read the content of each book and add it to the docs list
for i in range(len(books_collection)):
    with open(os.path.join(mypath, books_collection[i]), 'r', encoding='utf-8') as file:
        docs.append(file.read())

# Tokenize the content of each book
docs = [wordpunct_tokenize(doc) for doc in docs]

# Initialize the list to store the results after removing stopwords
docs_with_stopping = []

for i in range(len(docs)):
    docs_with_stopping.append([])
    for k in range(len(docs[i])):
        if docs[i][k].lower() not in stop_words:  # Check if it is a stopword
            docs_with_stopping[i].append(docs[i][k].lower())  # Store the word in lowercase

# Build the inverted index
for i in range(len(docs_with_stopping)):  # For every book
    count_term = dict()  # Create a temporary dictionary
    for term in docs_with_stopping[i]:
        if term not in count_term:  # If word not in temporary dictionary
            count_term[term] = 1
        else:
            count_term[term] += 1
    for term in count_term:  # For every word in the temporary dictionary
        if term not in the_inverted_index:  # If not in final index
            the_inverted_index[term] = []
        the_inverted_index[term].append((i, count_term[term]))  # Insert the results

# User search phrase
user_phrase = "Best places for programmers all around the world"  # Searching Phrase
user_phrase_token = wordpunct_tokenize(user_phrase)  # Tokenize
user_phrase_token = [word for word in user_phrase_token if not word in stop_words]  # Check for stopwords
user_phrase_token = [word.lower() for word in user_phrase_token]  # Convert to lowercase

# Using BM25 library to rank results
bm25 = BM25Okapi(docs_with_stopping)
doc_scores = bm25.get_scores(user_phrase_token)  # List with rank score for every book

# Writing the results to a .txt file
try:
    with open("results.txt", "w", encoding='utf-8') as f:
        for key, value in the_inverted_index.items():
            try:
                f.write(str(key) + str(value) + "\n")
            except UnicodeEncodeError:
                print("Can't encode the word: " + str(key) + str(value))
except Exception as e:
    print(f"Error at opening the file: {e}")

# Sort and print the final scores
final_scores = []
for i in range(len(doc_scores)):
    final_scores.append((i, doc_scores[i]))

final_scores.sort(reverse=True, key=lambda x: x[1])
print(final_scores)  # Result is a sorted list with tuples (DocId, RankBm25 score)

# Print runtime of the program
end = time.time()
print(f"Runtime of the program is {end - start}")

[(1, -0.5857246240799496), (0, -0.6269517981612489)]
Runtime of the program is 3.090391159057617
